In [4]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module

BATCH_SIZE = 100
WEIGHT_DECAY = 0.005
LEARNING_RATE = 0.0001
EPOCH = 100
PATH = "C:\\Users\\administrator\\CNNtest"

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root = PATH, train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE,
                        shuffle = True, num_workers = 2)

testset = torchvision.datasets.MNIST(root = PATH, train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 2)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, stride=2)

        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)

        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


device = torch.device("cuda:0")
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=WEIGHT_DECAY)

train_loss_value=[]      #trainingのlossを保持するlist
train_acc_value=[]       #trainingのaccuracyを保持するlist
test_loss_value=[]       #testのlossを保持するlist
test_acc_value=[]        #testのaccuracyを保持するlist 

for epoch in range(EPOCH):
    print('epoch', epoch+1)    #epoch数の出力
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    sum_loss = 0.0          #lossの合計
    sum_correct = 0         #正解率の合計
    sum_total = 0           #dataの数の合計

    #train dataを使ってテストをする(パラメータ更新がないようになっている)
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        sum_loss += loss.item()                            #lossを足していく
        _, predicted = outputs.max(1)                      #出力の最大値の添字(予想位置)を取得
        sum_total += labels.size(0)                        #labelの数を足していくことでデータの総和を取る
        sum_correct += (predicted == labels).sum().item()  #予想位置と実際の正解を比べ,正解している数だけ足す
    print("train mean loss={}, accuracy={}"
            .format(sum_loss*BATCH_SIZE/len(trainloader.dataset), float(sum_correct/sum_total))) #lossとaccuracy出力
    train_loss_value.append(sum_loss*BATCH_SIZE/len(trainloader.dataset))  #traindataのlossをグラフ描画のためにlistに保持
    train_acc_value.append(float(sum_correct/sum_total))   #traindataのaccuracyをグラフ描画のためにlistに保持

    sum_loss = 0.0
    sum_correct = 0
    sum_total = 0

    #test dataを使ってテストをする
    for (inputs, labels) in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        sum_loss += loss.item()
        _, predicted = outputs.max(1)
        sum_total += labels.size(0)
        sum_correct += (predicted == labels).sum().item()
    print("test  mean loss={}, accuracy={}"
            .format(sum_loss*BATCH_SIZE/len(testloader.dataset), float(sum_correct/sum_total)))
    test_loss_value.append(sum_loss*BATCH_SIZE/len(testloader.dataset))
    test_acc_value.append(float(sum_correct/sum_total))

plt.figure(figsize=(6,6))      #グラフ描画用

#以下グラフ描画
plt.plot(range(EPOCH), train_loss_value)
plt.plot(range(EPOCH), test_loss_value, c='#00ff00')
plt.xlim(0, EPOCH)
plt.ylim(0, 2.5)
plt.xlabel('EPOCH')
plt.ylabel('LOSS')
plt.legend(['train loss', 'test loss'])
plt.title('loss')
plt.savefig("loss_image.png")
plt.clf()

plt.plot(range(EPOCH), train_acc_value)
plt.plot(range(EPOCH), test_acc_value, c='#00ff00')
plt.xlim(0, EPOCH)
plt.ylim(0, 1)
plt.xlabel('EPOCH')
plt.ylabel('ACCURACY')
plt.legend(['train acc', 'test acc'])
plt.title('accuracy')
plt.savefig("accuracy_image.png")

epoch 1
train mean loss=2.2717865816752116, accuracy=0.25748333333333334
test  mean loss=2.2711802887916566, accuracy=0.265
epoch 2
train mean loss=2.2268558752536776, accuracy=0.5092666666666666
test  mean loss=2.2250664114952086, accuracy=0.5211
epoch 3
train mean loss=2.1409293591976164, accuracy=0.6042666666666666
test  mean loss=2.137463834285736, accuracy=0.6142
epoch 4
train mean loss=1.9479701954126358, accuracy=0.6427166666666667
test  mean loss=1.941026233434677, accuracy=0.6551
epoch 5
train mean loss=1.5159264918168385, accuracy=0.7236666666666667
test  mean loss=1.5014541065692901, accuracy=0.7389
epoch 6
train mean loss=0.973985844651858, accuracy=0.7987833333333333
test  mean loss=0.9524833750724793, accuracy=0.8096
epoch 7
train mean loss=0.6746343939006328, accuracy=0.8402666666666667
test  mean loss=0.6534766587615013, accuracy=0.8511
epoch 8
train mean loss=0.5363863701124986, accuracy=0.8624333333333334
test  mean loss=0.5162734872102738, accuracy=0.8682
epoch 9
tra

train mean loss=0.09900650262832641, accuracy=0.9718666666666667
test  mean loss=0.08800871101208031, accuracy=0.975
epoch 69
train mean loss=0.09816042670048773, accuracy=0.9724
test  mean loss=0.08778382971184329, accuracy=0.9755
epoch 70
train mean loss=0.09761958750896156, accuracy=0.97255
test  mean loss=0.0873390414332971, accuracy=0.9748
epoch 71
train mean loss=0.09654843266432483, accuracy=0.9726833333333333
test  mean loss=0.08608924221945927, accuracy=0.9749
epoch 72
train mean loss=0.09570198568205039, accuracy=0.9731333333333333
test  mean loss=0.08526023256825283, accuracy=0.9755
epoch 73
train mean loss=0.0953277642497172, accuracy=0.9729
test  mean loss=0.08504375219345092, accuracy=0.9758
epoch 74
train mean loss=0.09448382336025436, accuracy=0.97325
test  mean loss=0.08389450697926805, accuracy=0.9762
epoch 75
train mean loss=0.09371428409901758, accuracy=0.9736166666666667
test  mean loss=0.08436194281326607, accuracy=0.9753
epoch 76
train mean loss=0.093027645889669